In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"D:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|████████████████████████████████████████████████████████████████████████████| 10450/10450 [02:24<00:00, 72.18it/s]


Total cost = 1067.5922677892722

Cluster 74 assigned to Class 8. Cost: 2.117195653554159
Cluster 47 assigned to Class 59. Cost: 2.33011113871924
Cluster 368 assigned to Class 60. Cost: 2.8562288015500914
Cluster 396 assigned to Class 89. Cost: 2.4442838498253763
Cluster 118 assigned to Class 90. Cost: 4.795300675199588
Cluster 29 assigned to Class 95. Cost: 2.664792389993077
Cluster 350 assigned to Class 103. Cost: 4.705680827466046
Cluster 377 assigned to Class 109. Cost: 2.9779453915542544
Cluster 365 assigned to Class 111. Cost: 3.7558416997206745
Cluster 69 assigned to Class 117. Cost: 1.7973106593613009
Cluster 195 assigned to Class 119. Cost: 1.6126642429160922
Cluster 349 assigned to Class 129. Cost: 1.2107202716795435
Cluster 206 assigned to Class 137. Cost: 1.6525817118599835
Cluster 77 assigned to Class 192. Cost: 2.3993923194446665
Cluster 181 assigned to Class 230. Cost: 2.627946390408565
Cluster 256 assigned to Class 239. Cost: 1.8624384001997967
Cluster 258 assigned to Cl

Cluster 116 assigned to Class 4434. Cost: 1.2202467221918964
Cluster 291 assigned to Class 4497. Cost: 3.7728306692436817
Cluster 224 assigned to Class 4530. Cost: 2.828779771957585
Cluster 317 assigned to Class 4707. Cost: 0.9580863333916665
Cluster 398 assigned to Class 4975. Cost: 2.7803394632703737
Cluster 337 assigned to Class 4976. Cost: 1.935171106504236
Cluster 134 assigned to Class 4977. Cost: 2.5051511567212725
Cluster 363 assigned to Class 4990. Cost: 2.781605040880126
Cluster 99 assigned to Class 4997. Cost: 1.2806597290214752
Cluster 358 assigned to Class 4998. Cost: 4.960269203943205
Cluster 265 assigned to Class 5008. Cost: 4.891657400828163
Cluster 33 assigned to Class 5059. Cost: 3.281915953259212
Cluster 104 assigned to Class 5061. Cost: 2.2626163170216427
Cluster 55 assigned to Class 5142. Cost: 1.8375624722477388
Cluster 168 assigned to Class 5249. Cost: 4.926262493495131
Cluster 28 assigned to Class 5253. Cost: 4.803264209873077
Cluster 119 assigned to Class 5497. 

Cluster 252 assigned to Class 9343. Cost: 2.9038152593162327
Cluster 294 assigned to Class 9344. Cost: 2.1614205821970613
Cluster 125 assigned to Class 9346. Cost: 2.3821906171113456
Cluster 131 assigned to Class 9347. Cost: 1.2056885388646932
Cluster 322 assigned to Class 9359. Cost: 2.6435876862595102
Cluster 218 assigned to Class 9366. Cost: 2.916342171009594
Cluster 290 assigned to Class 9392. Cost: 1.6175295371603011
Cluster 387 assigned to Class 9396. Cost: 3.4906822300684293
Cluster 97 assigned to Class 9397. Cost: 3.4631469035348217
Cluster 389 assigned to Class 9429. Cost: 2.6933260482136525
Cluster 233 assigned to Class 9432. Cost: 1.6708502442828137
Cluster 8 assigned to Class 9436. Cost: 3.3042357172450227
Cluster 306 assigned to Class 9442. Cost: 2.6155393339462694
Cluster 49 assigned to Class 9444. Cost: 3.1820217641366697
Cluster 103 assigned to Class 9447. Cost: 8.229729218604689
Cluster 307 assigned to Class 9450. Cost: 3.790741522740424
Cluster 100 assigned to Class 9

In [6]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
